In [ ]:
!pip install transformers
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install easyocr
!sudo apt-get install -y poppler-utils
!pip install pdf2image
!wget "https://fonts.google.com/download?family=Roboto" -O roboto.zip
!unzip roboto.zip -d ./roboto

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 3s (1,666 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [ ]:
from transformers import AutoImageProcessor, TableTransformerForObjectDetection
import torch
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import csv
import numpy as np
import pandas as pd
from pdf2image import convert_from_path
from tqdm.auto import tqdm
import pytesseract
import easyocr

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pdf2image import convert_from_path
def pdf_to_img(pdf_path):
    # Convert all pages of the PDF to a list of images
    images = convert_from_path(pdf_path, fmt='jpeg')
    return images

In [ ]:

model_name = "microsoft/table-transformer-detection"
# define image preprocessor for table transformer
image_processor = AutoImageProcessor.from_pretrained(model_name)

# import table transformer model for table detection
model = TableTransformerForObjectDetection.from_pretrained(model_name,
                                                           revision="no_timm")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/273 [00:00<?, ?B/s]

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


config.json:   0%|          | 0.00/76.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

In [ ]:
def detect_table(image_doc):
    results = []

    for image in image_doc:
        # preprocess image document
        inputs = image_processor(images=image, return_tensors="pt")

        # detect tables
        outputs = model(**inputs)

        # convert outputs (bounding boxes and class logits) to Pascal VOC format (xmin, ymin, xmax, ymax)
        target_sizes = torch.tensor([image.size[::-1]])
        page_results = image_processor.post_process_object_detection(outputs,
                                                                     threshold=0.8,
                                                                     target_sizes=target_sizes)

        results.append({"image" : image ,"Result" : page_results})

    return results


In [ ]:
def get_table_bbox(results, label_map):
    """
    Extracts the bounding box coordinates for tables from detection results.

    Parameters:
    - results: List of dictionaries, each containing detection results for each page
    - label_map: Dictionary mapping label IDs to human-readable labels

    Returns:
    - tables_coordinates: List of dictionaries with bounding box coordinates in Pascal VOC format
    """
    tables_coordinates = []

    for page_idx, page_results in enumerate(results):
        image = page_results["image"]
        for detection in page_results["Result"]:
            boxes = detection["boxes"]
            scores = detection["scores"]
            labels = detection["labels"]

            for score, label, box in zip(scores, labels, boxes):
                if score >= 0.95:  # Only consider detections with high confidence
                    # Convert box coordinates to integer and ensure Pascal VOC format
                    xmin, ymin, xmax, ymax = [round(i, 2) for i in box.tolist()]

                    table_dict = {
                        "page": page_idx + 1,
                        "label": label_map.get(label.item(), "Unknown"),
                        "confidence": round(score.item(), 3),
                        "xmin": xmin,
                        "ymin": ymin,
                        "xmax": xmax,
                        "ymax": ymax
                    }

                    tables_coordinates.append(table_dict)

                    # Print prediction label, confidence score, and bbox values
                    print(
                        f"Page {page_idx + 1}: Detected {label_map.get(label.item(), 'Unknown')} with confidence "
                        f"{table_dict['confidence']} at location (xmin: {xmin}, ymin: {ymin}, xmax: {xmax}, ymax: {ymax})"
                    )

    return tables_coordinates

In [ ]:
from PIL import ImageDraw

def highlight_tables(image, table_boxes, padding):
    """
    Highlights the detected tables on the image with red rectangles.

    Parameters:
    - image: The PIL.Image object to highlight
    - table_boxes: List of dictionaries with table bounding box coordinates
    - padding: Padding to add around each bounding box

    Returns:
    - doc_image: The image with highlighted tables
    """
    # Create a drawing context for the image
    doc_image = image.copy()
    draw = ImageDraw.Draw(doc_image)

    # Iterate over each table bounding box
    for table in table_boxes:
        # Define the coordinates for the rectangle with padding
        rectangle_coords = (table["xmin"] - padding,
                            table["ymin"] - padding,
                            table["xmax"] + padding,
                            table["ymax"] + padding)

        # Draw a red rectangle around the detected table
        draw.rectangle(rectangle_coords, outline="red", width=2)

    return doc_image



In [ ]:
from PIL import Image

def get_cropped_images(image, table_bbox, padding):
    """
    Crops out regions corresponding to detected tables from the image.

    Parameters:
    - image: The PIL.Image object from which tables are to be cropped
    - table_bbox: List of dictionaries with bounding box coordinates for each table
    - padding: Padding to apply around each bounding box

    Returns:
    - cropped_images: List of cropped images (PIL.Image objects) for each detected table
    """
    cropped_images = []

    if not table_bbox:
        print("No tables detected in the image.")
        return cropped_images

    for table in table_bbox:
        # Calculate coordinates with padding
        left = max(table["xmin"] - padding, 0)
        top = max(table["ymin"] - padding, 0)
        right = min(table["xmax"] + padding, image.width)
        bottom = min(table["ymax"] + padding, image.height)

        # Crop the image
        cropped_image = image.crop((left, top, right, bottom))
        cropped_images.append(cropped_image)

    return cropped_images


In [ ]:

# import model for detecting table features e.g. rows, columns, etc
structure_model = TableTransformerForObjectDetection.from_pretrained("microsoft/table-structure-recognition-v1.1-all")


config.json:   0%|          | 0.00/76.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

In [ ]:

def get_table_features(cropped_image):
    """
    Extract features from a cropped image of a table.

    Parameters:
    - cropped_image: The PIL.Image object representing a cropped table

    Returns:
    - features: List of dictionaries, each containing detected feature label, confidence score, and bbox values
    """
    # Preprocess image input for the table structure model
    inputs = image_processor(images=cropped_image, return_tensors="pt")

    # Make prediction using the table structure model
    with torch.no_grad():
        outputs = structure_model(**inputs)

    # Post-process output to get bounding box format in Pascal VOC
    target_sizes = torch.tensor([cropped_image.size[::-1]])
    results = image_processor.post_process_object_detection(outputs, threshold=0.9, target_sizes=target_sizes)[0]

    # List to store detected features
    features = []

    # Iterate through all detected features
    for i, (score, label, box) in enumerate(zip(results["scores"], results["labels"], results["boxes"])):
        # Convert the box tensor to a list of rounded floats
        box = [round(coord, 2) for coord in box.tolist()]
        score = score.item()
        label = structure_model.config.id2label[label.item()]

        # Create a dictionary for each detected feature
        feature_dict = {
            "label": label,
            "score": score,
            "bbox": box
        }

        # Append to the features list
        features.append(feature_dict)

        # Print detected features
       # print(f"Detected {label} with confidence {round(score, 3)} at location {box}")

    return features



In [ ]:
from PIL import ImageDraw, ImageFont
def display_detected_features(cropped_image, features):
    """
    Draws bounding boxes and labels on a cropped image based on detected features.

    Parameters:
    - cropped_image: The PIL.Image object representing a cropped table
    - features: List of dictionaries containing feature label, confidence score, and bbox values

    Returns:
    - cropped_table_visualized: The image with bounding boxes and labels drawn
    """
    cropped_table_visualized = cropped_image.copy()
    draw = ImageDraw.Draw(cropped_table_visualized)

    # Load a font; replace with a valid path to a .ttf file on your system
    try:
        font_path = "/content/drive/MyDrive/Roboto-Italic.ttf"
        font = ImageFont.truetype(font_path, 15)
    except IOError:
        print(f"Could not load font from {font_path}. Using default font.")
        font = ImageFont.load_default()

    # Iterate through all features and display bounding boxes with text labels
    for feature in features:
        # Draw rectangle for each feature's bounding box
        draw.rectangle(feature["bbox"], outline="red", width=2)

        # Calculate text position and draw the label
        text_position = (feature["bbox"][0], max(feature["bbox"][1] - 15, 0))
        draw.text(text_position, feature["label"], fill="blue", font=font)

    return cropped_table_visualized





In [ ]:
def get_cell_coordinates_by_row(table_data):
    # Extract rows and columns
    rows = [entry for entry in table_data if entry['label'] == 'table row']
    columns = [entry for entry in table_data if entry['label'] == 'table column']

    # Sort rows and columns by their Y and X coordinates, respectively
    rows.sort(key=lambda x: x['bbox'][1])
    columns.sort(key=lambda x: x['bbox'][0])


    # Initialize cell coordinates list
    cell_coordinates = []

    # Generate cell coordinates for each row
    for row in rows:
        row_cells = []
        for column in columns:
            # Calculate cell bounding box
            cell_bbox = [column['bbox'][0], row['bbox'][1], column['bbox'][2], row['bbox'][3]]
            row_cells.append({'cell': cell_bbox})
        cell_coordinates.append({'cells': row_cells, 'cell_count': len(row_cells)})

    return cell_coordinates




In [ ]:

def apply_ocr(cell_coordinates, cropped_image):
    # let's OCR row by row
    data = dict()
    max_num_columns = 0
    for idx, row in enumerate(tqdm(cell_coordinates)):
        row_text = []
        for cell in row["cells"]:
            # crop cell out of image
            cell_image = np.array(cropped_image.crop(cell["cell"]))

            # apply OCR using PyTesseract
            text = pytesseract.image_to_string(cell_image, lang='eng', config='--psm 6').strip()
            if text:
                row_text.append(text)


        if len(row_text) > max_num_columns:
            max_num_columns = len(row_text)

        data[idx] = row_text

    print("Max number of columns:", max_num_columns)

    # pad rows which don't have max_num_columns elements
    for row, row_data in data.copy().items():
        if len(row_data) != max_num_columns:
            row_data = row_data + ["" for _ in range(max_num_columns - len(row_data))]
        data[row] = row_data
        print(row_data)

    return data



In [ ]:
import csv
import pandas as pd

def write_csv(data):
  with open('output.csv','w') as result_file:
      wr = csv.writer(result_file, dialect='excel')
      # Verify if 'data' is indeed a dictionary
      if isinstance(data, dict):
          for row, row_text in data.items():
              wr.writerow(row_text)
      else:
          print("Error: 'data' is not a dictionary. Cannot write to CSV.")


In [ ]:
import os
import pandas as pd
import gc
from tqdm import tqdm
import json
from pdf2image import convert_from_path
from transformers import AutoImageProcessor, TableTransformerForObjectDetection

# Function to convert PDF to images
def pdf_to_img(pdf_path):
    images = convert_from_path(pdf_path, fmt='jpeg')
    return images

# Function to detect tables in a batch of images
def detect_table(image_doc):
    results = []
    for image in image_doc:
        inputs = image_processor(images=image, return_tensors="pt")
        outputs = model(**inputs)
        target_sizes = torch.tensor([image.size[::-1]])
        page_results = image_processor.post_process_object_detection(outputs, threshold=0.9, target_sizes=target_sizes)
        results.append({"image": image, "Result": page_results})
    return results

# Function to extract bounding boxes for detected tables
def get_table_bbox(results, label_map):
    tables_coordinates = []
    for page_idx, page_results in enumerate(results):
        for detection in page_results["Result"]:
            boxes = detection["boxes"]
            scores = detection["scores"]
            labels = detection["labels"]
            for score, label, box in zip(scores, labels, boxes):
                if score >= 0.96:
                    xmin, ymin, xmax, ymax = [round(i, 2) for i in box.tolist()]
                    table_dict = {
                        "page": page_idx + 1,
                        "label": label_map.get(label.item(), "Unknown"),
                        "confidence": round(score.item(), 3),
                        "xmin": xmin,
                        "ymin": ymin,
                        "xmax": xmax,
                        "ymax": ymax
                    }
                    tables_coordinates.append(table_dict)
    return tables_coordinates

# Function to process a batch of pages
def process_batch(batch_images, batch_num):
    table_results = detect_table(batch_images)
    label_map = {1: 'table'}
    table_bbox = get_table_bbox(table_results, label_map)
    padding = 40
    all_cropped_images = []
    for i, image in enumerate(batch_images):
        page_num =  i + 1
        table_boxes = [table for table in table_bbox if table["page"] == page_num]
        cropped_images = get_cropped_images(image, table_boxes, padding)
        all_cropped_images.extend(cropped_images)
    tables_features = {}
    for idx, cropped_image in enumerate(all_cropped_images):
        features = get_table_features(cropped_image)
        tables_features[idx] = {
            "image": cropped_image,
            "features": features
        }
    all_table_coordinates = []
    for table_id, data in tables_features.items():
        table_coordinates = get_cell_coordinates_by_row(data["features"])
        all_table_coordinates.append({"ID": table_id, "Coordinates": table_coordinates})
    for table_coordinate in tqdm(all_table_coordinates):
        table_data = apply_ocr(table_coordinate['Coordinates'], all_cropped_images[table_coordinate['ID']])
        all_table_data.append({
            "Batch": batch_num,
            "TableID": table_coordinate['ID'],
            "TableData": dataframe_to_json(table_data)
        })

# Function to convert table data to JSON
def dataframe_to_json(table_data):
    df = pd.DataFrame(table_data)
    return df.to_dict(orient='records')

# Initialize paths and parameters
input_folder = '/content/drive/MyDrive/PDF_FILES'  # Replace with your input folder path
output_folder = '/content/drive/MyDrive/output_Etracted'  # Replace with your output folder path

batch_size = 5
image_processor = AutoImageProcessor.from_pretrained("microsoft/table-transformer-detection")
model = TableTransformerForObjectDetection.from_pretrained("microsoft/table-transformer-detection", revision="no_timm")
all_table_data = []

# Ensure output folder exists, create if not
os.makedirs(output_folder, exist_ok=True)

# Process each PDF file in the input folder
for pdf_file in os.listdir(input_folder):
    if pdf_file.endswith('.pdf'):
        pdf_path = os.path.join(input_folder, pdf_file)
        output_json_file = os.path.join(output_folder, f"{os.path.splitext(pdf_file)[0]}.json")

        # Convert PDF to images and get the total number of pages
        images = pdf_to_img(pdf_path)
        total_pages = len(images)
        num_batches = (total_pages + batch_size - 1) // batch_size

        # Process each batch
        for batch_num in range(num_batches):
            start_page = batch_num * batch_size
            end_page = min((batch_num + 1) * batch_size, total_pages)
            batch_images = images[start_page:end_page]

            # Process the current batch
            process_batch(batch_images, batch_num)

            # Clear variables and perform garbage collection to manage memory
            del batch_images
            gc.collect()

        # Save all tables to a JSON file
        with open(output_json_file, 'w') as f:
            json.dump(all_table_data, f, indent=4)

        print(f"All tables extracted and saved to {output_json_file}")


No tables detected in the image.
No tables detected in the image.
No tables detected in the image.


 50%|█████     | 1/2 [00:20<00:20, 20.51s/it]

Max number of columns: 3
['1.0', 'TNtrOductiOn..........ccccccccccsseccccuccececcueececccecceccuseccccuecccceuseecsssuuecsscsuuececessnsececsutessecseece', '']
['1.1.', 'Study ODjectives: ..........cccccccccccccccccccccccccceccccuececcecucecccccuccecsscuececcssueececsusscsceeueeeees', '']
['1.2.', 'Backeround And Rationale: ..............cccccccccccccccecccscceucccuccccscccusccuseceusccuscecuccenseeene', '']
['2.0', 'Eligibility Assessment and Enrollment............c...cccccccccccssecccccsceecececseccecscsecescansecceuans', '']
['2.1.', 'Eligibility Criteria 00... ..cccccccccccccccccceccccccecccccucecccccsececcuceccecauseccesucseccecsesescesaueecees', '']
['2.2.', 'Screening Evaluation ..........cccccccccccscccccecceececeeccccecceececsseececeurececseseessssstsececuuneeeess', '']
['2.3.', 'Registration PLOC@CUIESS ................cccssssssseesseecececcecccccecccccucasaessssesecesecececececeeeceeas', '']
['3.0', 'Study Implementation .....ccccccccccsccscscsececececesceseccevscescssssevsc


100%|██████████| 2/2 [00:27<00:00, 13.61s/it]


Max number of columns: 3
['8.6.', '. Trimethoprim and Sulfamethoxazole Double Strensth (TMP / SMX DS) .........', 'eceeeee 47']
['8.7.', '. Herpes Virus Prophylaxis ccvavavavavavavavavavavavavavavavavevavavevavavevavavavevavevavavevatevevevevivesesesees', 'cose AY']
['8.8.', '© FUCOMA TON oo... ceccccecececcccosscsccecscsesecscsecsvscssvsvssssvevsessvevessvsvevsvisevsvssisvevsvisusvavssssvevsvesees', 'cesses. 48']
['8.9.', '. Anti-VEGFR2 CAR transduced CD8+ PBL .............ccccccccccccccccccccccccccecccecseeccceeusecs', 'esseeee 48']
['8.10', '0. Support Medications......................:ccccceseecccceccccccccccccccccsssseseesececececececcececeescsaaaaaseeess', 'veeeeeee 49']
['9.0', 'PRTC encase nc cc cee ss en se case cede ee nee eon ca esd saci ance densa aca cists vei hese dgawstc bees', 'arsssenn OU']
['10.0', '‘Tables: Frewres. and AppendiCes!vescenessssesesssmeanceeneeeese eee ere', 'saree 9']
No tables detected in the image.
No tables detected in the image.
No tables detected 

...

No tables detected in the image.
No tables detected in the image.
No tables detected in the image.


 50%|█████     | 1/2 [00:06<00:06,  6.33s/it]

Max number of columns: 3
['REGIMEN', 'DRUG', 'TREATMENT']
['lfosfamide', '5000 mg/m2 mixed with Mesna 5000 mg/m2 iv over 24 hrs d2', '']
['ICE regimen 2', 'Carboplatin', '| AUC 5 (max 800mg) iv d2']
['Q2w x 3 cycles', 'Etoposide', '100 mg/m2/d iv d1-3']
['fo', 'G-CSF 5 ug/kg sc qd d5-12', '']
['Rituximab', '375 mg/m2 iv d1 q2w x 3 cycles', '']
['RICE', 'lfosfamide', '5000 mg/m2 mixed with Mesna 5000 mg/m2 iv over 24 hrs d2']
['Oe i et', 'Carboplatin', 'AUC 5 (max 800mg) iv d2']
['EER: eR, ee', 'Etoposide', '100 mg/m2/d iv d1-3']
['fo', 'G-CSF 5 ug/kg sc qd d5-12', '']
['ESHAP', 'Etoposide', '40 ma/m2/d iv over 1 hr d1-4']



 50%|█████     | 1/2 [00:08<00:08,  8.32s/it]


KeyboardInterrupt: 